In [1]:
#Importe
import twitter
import time
import csv

# Twitter API *#apple*
Für den Zugriff auf die Twitter API benutzen wir das Package *python-twitter*. Gesammelt werden sollen englischsprachige Tweets die *"#apple"* enthalten.<br>
Zuerst müssen wir uns bei der Twitter API mit den im Twitter Developer Account generierten Anmeldedaten registrieren.

In [2]:
api = twitter.Api(consumer_key='dummy',
                  consumer_secret='dummy',
                  access_token_key='dummy',
                  access_token_secret='dummy',
                  tweet_mode="extended") #Tweets sollen nicht gekürzt werden

Die Suchanfrage *search* sucht nach dem neusten Tweet der das Keyword *"#apple"* enthält. Retweets werden bei der Suche ausgeschlossen.<br>
Falls die Suche fehlschlägt wird jede 5 Sekunden eine neue Suchanfrage gestartet, um nicht die maximal zulässige Anzahl der Zugriffe auf die Twitter API von 150 Zugriffen pro 15 Minuten zu überschreiten.

In [3]:
search = api.GetSearch(raw_query="f=tweets&vertical=news&q=%23apple-filter%3Aretweets&count=1")

while not search:
    time.sleep(5)
    search = api.GetSearch(raw_query="f=tweets&vertical=news&q=%23apple-filter%3Aretweets&count=1")

Nun muss noch einmal die Sprache des Tweets überprüft werden, da eine Query mit Language Tag "*en*" trotzdem Tweets in allen Sprachen liefert. Falls der Language Tag des Tweets nicht "*en*" ist wird deshalb solange eine neue Suchanfrage gestartet bis es sich um einen englischsprachigen Tweet handelt.

In [4]:
while True:
    try:
        if search[0].lang == "en":
            break
        else:
            time.sleep(5)
            search = api.GetSearch(raw_query="f=tweets&vertical=news&q=%23apple-filter%3Aretweets&count=1")
            continue
    except:
        time.sleep(5)
        continue

Die Query liefert ein Objekt mit unzähligen verschiedenen Parametern des Tweets, wir benötigen aber nur das Datum an dem er gepostet wurde *"created_at"* und den Tweet selbst *"full_text"* ohne Zeilenumbrüche. Beide Einträge werden in eine csv-Datei geschrieben.

In [ ]:
csv.register_dialect('quotes', quotechar = '"', quoting=csv.QUOTE_ALL)
with open("data/raw/dataset_apple2.csv", "a", newline="", encoding='utf8') as file:
    file_writer = csv.writer(file, dialect='quotes')
    file_writer.writerow([search[0].created_at, search[0].full_text.replace("\n", " ")])

Nun startet eine Endlosschleife, die jede 5 Sekunden nach dem neusten Tweet sucht und ihn mit dem letzen gefundenen vergleicht. Falls die Tweets nicht gleich sind, also inzwischen ein neuer Tweet gepostet wurde, wird der Tweet mit Datum in die csv-Datei geschrieben.

In [ ]:
while True:
    try:
        time.sleep(5)
        refresh = api.GetSearch(raw_query="f=tweets&vertical=news&q=%23apple-filter%3Aretweets&count=1")

        if not refresh:
            continue

        if search[0].full_text != refresh[0].full_text and refresh[0].lang == "en":
            search = refresh
            with open("data/raw/dataset_apple2.csv", "a", newline="", encoding='utf8') as file:
                file_writer = csv.writer(file, dialect='quotes')
                file_writer.writerow([search[0].created_at, search[0].full_text.replace("\n", " ")])
            print(search[0].full_text.replace("\n", " "))
    except:
        time.sleep(5)
        continue

Belgian Apple Store is back online! Time to checkout the changes #apple #store #semonto  #1902040358
